# TCGA clinical xml processing

In [1]:
import os, glob
import pandas as pd
import xml.etree.ElementTree as ET
import collections

In [2]:
path = '/csb1/project/JingSY_GasSignature_2023/RAWDATA/0_clinical/TCGA-COAD/'
xmlList = glob.glob(path + '*/*.xml')
print(f"find {len(xmlList)} xml")

find 459 xml


In [3]:
# 提取节点信息
def extract_nodeInfo(node):
    _, tag = node.tag.rsplit("}", 1)
    text = node.text
    return tag, text

In [4]:
# 遍历所有节点
def traverse_node(node):
    tagList, textList = [], []
    for child in node:
        tag, text = extract_nodeInfo(child)
        tagList.append(tag)
        textList.append(text)
        childRestag, childRestext = traverse_node(child) # child 的子节点
        
        if childRestag : # 将子节点的子节点存储到父节点的字典中
            tagList.append(childRestag)
            textList.append(childRestext)
    return tagList, textList

In [5]:
# 列表展平
def flatten_list(lst):
    flattened = []
    for item in lst:
        if isinstance(item, list):
            flattened.extend(flatten_list(item))
        else:
            flattened.append(item)
    return flattened

In [6]:
for i, filePath in enumerate(xmlList): # 1 patient
    dataRow = {}
    
    tree = ET.parse(filePath)
    root = tree.getroot() # root：tcga_bcr
    info1 = root[0] # tcga_bcr/admin/*
    info2 = root[1] # tcga_bcr/patient/*
    tagList, texttList = traverse_node(info2)
    
    tagFlatten = flatten_list(tagList)
    textFlatten = flatten_list(texttList)
    
    dataRow = {x: y for x, y in zip(tagFlatten, textFlatten)}
    
    if i ==0:
        df = pd.DataFrame(dataRow, index=[i]) #首行
    else:
        df = df.append(dataRow, ignore_index=True)

/tmp/ipykernel_125896/1349098377.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(dataRow, ignore_index=True)
/tmp/ipykernel_125896/1349098377.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(dataRow, ignore_index=True)
/tmp/ipykernel_125896/1349098377.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(dataRow, ignore_index=True)
/tmp/ipykernel_125896/1349098377.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(dataRow, ignore_index=True)
/tmp/ipykernel_125896/1349098377.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in 

In [7]:
df.to_csv('clinical_data.csv', index=False)